In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import Model, Sequential, load_model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


Read in the metadata.

In [2]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

Set the main constants.

In [3]:
INPUT_SHAPE = (256, 384, 3)

LEARNING_RATE = 0.0001
N_EPOCHS = 100
BATCH_SIZE = 32

Create the f1 metric to be used during model training.

In [4]:
class F1_Metric(Callback):
    def on_train_begin(self, logs={}):
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs={}):
        #score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]

        self.f1_scores.append(f1_score(targ, predict, average='weighted'))
        
        print(f' val_f1: {self.f1_scores[-1]}')
        
        return
    
f1_metric = F1_Metric()

Compute the class weights to be used during model training in order to fight the class imbalances.

In [5]:
class_values = train_set['category'].values - 1

classes = np.unique(class_values)
weights = compute_class_weight('balanced', np.unique(class_values), class_values)

class_weights = dict(zip(classes, weights))
class_weights

{0: 0.589811320754717,
 1: 1.0717714285714286,
 2: 1.3650655021834062,
 3: 1.5028846153846154,
 4: 1.0271631982475355}

Read in the best model with respect to the f1 score obtained in the first part of the model training. Unfreeze the last 13 layers and compile the new model.

In [6]:
model = load_model('model_006_epoch_088_saved.hdf5')

for layer in model.layers[-13:]:
    layer.trainable = True

for layer in model.layers:
    print(layer.trainable)

# compile the model to reflect the above changes
adam = Adam(lr=LEARNING_RATE)
model.compile(optimizer=adam, loss='categorical_crossentropy')    

model.summary()

False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 384, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 384, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 384, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 192, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 192, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 192, 128)     147584    
___________________________________

Train the model utilizing Adam optimizer, learning rate reduction on plateau, class weights and data augmentation. *Note that the learning was manually stopped after 50 epochs*.

In [7]:
# Prepare the training and validation data
X_train = np.load('train_set_hmgd_arr_256_384_VGG16.npy')
y_train = to_categorical(train_set['category'].values - 1)

X_valid = np.load('valid_set_hmgd_arr_256_384_VGG16.npy')
y_valid = to_categorical(valid_set['category'].values - 1)


# Initialize callbacks
checkpoint = ModelCheckpoint('model_010_FT001_epoch_{epoch:03d}.hdf5', monitor='val_loss', 
                             save_best_only=False, save_weights_only=False)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
#early_stopping = EarlyStopping(monitor='val_loss', patience=12)    
callback_list = [f1_metric, checkpoint, lr_reduction]#, early_stopping]


# Fit the model
#model.fit(X_train, y_train, epochs=N_EPOCHS, batch_size=BATCH_SIZE, class_weight=class_weights, 
#          callbacks=callback_list, validation_data=(X_valid, y_valid))

data_gen = ImageDataGenerator(rotation_range=20, 
                              width_shift_range=0.2, 
                              height_shift_range=0.2, 
                              horizontal_flip=True)

steps_per_epoch = int(len(X_train)) / BATCH_SIZE

model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=N_EPOCHS,
                    callbacks=callback_list,
                    validation_data=(X_valid, y_valid),
                    class_weight=class_weights,
                    workers=4, 
                    verbose=2)

Epoch 1/100
 - 212s - loss: 0.8147 - val_loss: 0.5659
 val_f1: 0.7409163999894015
Epoch 2/100
 - 188s - loss: 0.4328 - val_loss: 0.3066
 val_f1: 0.8883894859957736
Epoch 3/100
 - 188s - loss: 0.2991 - val_loss: 0.2938
 val_f1: 0.8882914093962836
Epoch 4/100
 - 188s - loss: 0.3019 - val_loss: 0.3855
 val_f1: 0.864367028484374
Epoch 5/100
 - 188s - loss: 0.2491 - val_loss: 0.2420
 val_f1: 0.9131002905905476
Epoch 6/100
 - 188s - loss: 0.2454 - val_loss: 0.4758
 val_f1: 0.8622775099954675
Epoch 7/100
 - 188s - loss: 0.2507 - val_loss: 0.4666
 val_f1: 0.8441494555096982
Epoch 8/100
 - 188s - loss: 0.1729 - val_loss: 0.2211
 val_f1: 0.913030647336355
Epoch 9/100
 - 187s - loss: 0.1430 - val_loss: 0.2642
 val_f1: 0.9126266592400836
Epoch 10/100
 - 188s - loss: 0.1729 - val_loss: 0.2393
 val_f1: 0.9175662696639433
Epoch 11/100
 - 187s - loss: 0.1594 - val_loss: 0.2903
 val_f1: 0.9185588229494767
Epoch 12/100
 - 187s - loss: 0.1215 - val_loss: 0.2257
 val_f1: 0.9240445515995709
Epoch 13/100
 -

KeyboardInterrupt: 

Free up memory and read in the test data.

In [8]:
del X_train
del X_valid

X_test = np.load('test_set_hmgd_arr_256_384_VGG16.npy')

Read in the best 2 models with respect to the f1 score.

In [10]:
model_01 = load_model('model_010_FT001_epoch_023.hdf5')
model_02 = load_model('model_010_FT001_epoch_046.hdf5')

Compute the predictions and prepare the submissions.

In [11]:
sample_submission = pd.read_csv('data/sample_submission.csv')
submission = sample_submission.copy()

In [12]:
pred_prob_01 = model_01.predict(X_test)
pred_prob_02 = model_02.predict(X_test)

In [13]:
pred_classes_01 = pred_prob_01.argmax(axis=-1) + 1
pred_classes_02 = pred_prob_02.argmax(axis=-1) + 1

In [14]:
submission['category'] = pred_classes_01
submission['category'].value_counts()

1    913
2    516
5    512
3    389
4    350
Name: category, dtype: int64

In [15]:
submission.to_csv('model_010_submission_01.csv', index=False)

In [16]:
submission['category'] = pred_classes_02
submission['category'].value_counts()

1    926
5    515
2    503
3    391
4    345
Name: category, dtype: int64

In [17]:
submission.to_csv('model_010_submission_02.csv', index=False)

In [ ]:
#                      Epoch 23      Epoch 46
# Validation score:  0.9578207955  0.9577930088
# Leaderboard score: 0.9617234462  0.9609120265